In [1]:
import json 

In [4]:
body = json.loads(event['body'])

In [5]:
body['context']
body['question']

'What is the GLUE score for Bert?'

In [3]:
event = {"body":"{\"context\":\"We introduce a new language representation model called BERT, which stands for Bidirectional Encoder Representations from Transformers. Unlike recent language representation models (Peters et al., 2018a; Radford et al., 2018), BERT is designed to pretrain deep bidirectional representations from unlabeled text by jointly conditioning on both left and right context in all layers. As a result, the pre-trained BERT model can be finetuned with just one additional output layer to create state-of-the-art models for a wide range of tasks, such as question answering and language inference, without substantial taskspecific architecture modifications. BERT is conceptually simple and empirically powerful. It obtains new state-of-the-art results on eleven natural language processing tasks, including pushing the GLUE score to 80.5% (7.7% point absolute improvement), MultiNLI accuracy to 86.7% (4.6% absolute improvement), SQuAD v1.1 question answering Test F1 to 93.2 (1.5 point absolute improvement) and SQuAD v2.0 Test F1 to 83.1 (5.1 point absolute improvement).\",\n\"question\":\"What is the GLUE score for Bert?\"\n}"}

In [9]:
event['body']

'{"context":"We introduce a new language representation model called BERT, which stands for Bidirectional Encoder Representations from Transformers. Unlike recent language representation models (Peters et al., 2018a; Radford et al., 2018), BERT is designed to pretrain deep bidirectional representations from unlabeled text by jointly conditioning on both left and right context in all layers. As a result, the pre-trained BERT model can be finetuned with just one additional output layer to create state-of-the-art models for a wide range of tasks, such as question answering and language inference, without substantial taskspecific architecture modifications. BERT is conceptually simple and empirically powerful. It obtains new state-of-the-art results on eleven natural language processing tasks, including pushing the GLUE score to 80.5% (7.7% point absolute improvement), MultiNLI accuracy to 86.7% (4.6% absolute improvement), SQuAD v1.1 question answering Test F1 to 93.2 (1.5 point absolute 

In [5]:
event['question']

KeyError: 'question'

In [44]:
import json
import torch
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, AutoConfig

def encode(tokenizer, question, context):
    """encodes the question and context with a given tokenizer"""
    encoded = tokenizer.encode_plus(question, context)
    return encoded["input_ids"], encoded["attention_mask"]

def decode(tokenizer, token):
    """decodes the tokens to the answer with a given tokenizer"""
    answer_tokens = tokenizer.convert_ids_to_tokens(
        token, skip_special_tokens=True)
    return tokenizer.convert_tokens_to_string(answer_tokens)

def serverless_pipeline(model_path='./model'):
    """Initializes the model and tokenzier and returns a predict function that ca be used as pipeline"""
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForQuestionAnswering.from_pretrained(model_path)
    def predict(question, context):
        """predicts the answer on an given question and context. Uses encode and decode method from above"""
        input_ids, attention_mask = encode(tokenizer,question, context)
        a = model(torch.tensor(
            [input_ids]), attention_mask=torch.tensor([attention_mask]))
        start_scores = a['start_logits']
        end_scores = a['end_logits']
        ans_tokens = input_ids[torch.argmax(
            start_scores): torch.argmax(end_scores)+1]
        answer = decode(tokenizer,ans_tokens)
        return answer
    return predict

# initializes the pipeline
question_answering_pipeline = serverless_pipeline()


In [46]:
model_path='./model'
model = AutoModelForQuestionAnswering.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

input_ids, attention_mask = encode(tokenizer,body['question'], body['context'])
a = model(torch.tensor(
            [input_ids]), attention_mask=torch.tensor([attention_mask]))
start_scores = a['start_logits']
end_scores = a['end_logits']
ans_tokens = input_ids[torch.argmax(
            start_scores): torch.argmax(end_scores)+1]
answer = decode(tokenizer,ans_tokens)
answer

'80. 5 %'

In [52]:
print(torch.argmax(start_scores))
print(torch.argmax(end_scores)+1)

tensor(180)
tensor(184)


In [18]:
answer = question_answering_pipeline(question=body['question'], context=body['context'])

In [55]:
start_scores

tensor([[ -0.9935,  -9.6032, -12.6266,  -8.8786,  -9.7391, -10.4422, -13.1308,
         -10.5575, -13.7492, -11.6378,  -8.5799, -10.5312,  -9.8652,  -9.7580,
          -9.4949, -11.9562, -12.3860, -12.0936,  -8.6679, -12.0692, -10.2790,
         -10.8464, -12.4050,  -7.5609, -12.8723, -12.9467, -13.6367,  -9.9548,
         -12.5127, -13.8898, -10.5344, -12.8058,  -9.0189, -11.6492, -10.1382,
          -9.3935,  -9.7570, -12.3048, -12.3742, -10.0070,  -7.4577, -11.7353,
         -13.9000, -14.5311, -12.7638,  -8.2836, -13.4187, -11.9397,  -7.5125,
         -12.0522, -11.9577, -13.0190, -13.8692, -13.9177, -12.6028,  -8.6266,
         -12.6308, -11.2395,  -8.2875, -11.5319,  -9.7516,  -9.8484,  -8.8828,
         -12.1323,  -9.1497,  -9.5073, -14.3122, -14.7056, -15.2779, -12.3574,
         -12.2021,  -8.8719, -12.2928, -13.3568, -13.6476, -10.2934,  -9.7695,
          -8.4249,  -9.5299, -10.4064,  -8.4501,  -8.1455, -13.3144,  -9.5901,
          -9.9255,  -9.9359,  -8.9109, -10.1722, -10

In [54]:
input_ids

[101,
 2054,
 2003,
 1996,
 25238,
 3556,
 2005,
 14324,
 1029,
 102,
 2057,
 8970,
 1037,
 2047,
 2653,
 6630,
 2944,
 2170,
 14324,
 1010,
 2029,
 4832,
 2005,
 7226,
 7442,
 7542,
 2389,
 4372,
 16044,
 2099,
 15066,
 2013,
 19081,
 1012,
 4406,
 3522,
 2653,
 6630,
 4275,
 1006,
 12420,
 3802,
 2632,
 1012,
 1010,
 2760,
 2050,
 1025,
 10958,
 20952,
 8551,
 3802,
 2632,
 1012,
 1010,
 2760,
 1007,
 1010,
 14324,
 2003,
 2881,
 2000,
 3653,
 23654,
 2784,
 7226,
 7442,
 7542,
 2389,
 15066,
 2013,
 4895,
 20470,
 12260,
 2094,
 3793,
 2011,
 10776,
 14372,
 2006,
 2119,
 2187,
 1998,
 2157,
 6123,
 1999,
 2035,
 9014,
 1012,
 2004,
 1037,
 2765,
 1010,
 1996,
 3653,
 1011,
 4738,
 14324,
 2944,
 2064,
 2022,
 2986,
 8525,
 7228,
 2007,
 2074,
 2028,
 3176,
 6434,
 6741,
 2000,
 3443,
 2110,
 1011,
 1997,
 1011,
 1996,
 1011,
 2396,
 4275,
 2005,
 1037,
 2898,
 2846,
 1997,
 8518,
 1010,
 2107,
 2004,
 3160,
 10739,
 1998,
 2653,
 28937,
 1010,
 2302,
 6937,
 8518,
 5051,
 6895,
 88

In [22]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("hoanhkhoa/roberta-base-finetuned-ner")
model = AutoModelForTokenClassification.from_pretrained("hoanhkhoa/roberta-base-finetuned-ner")


In [23]:
tokenizer

PreTrainedTokenizerFast(name_or_path='hoanhkhoa/roberta-base-finetuned-ner', vocab_size=50265, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False)})

In [35]:
def encode(tokenizer, question):
    """encodes the question and context with a given tokenizer"""
    encoded = tokenizer.encode_plus(question)
    return encoded["input_ids"], encoded["attention_mask"]

def decode(tokenizer, token):
    """decodes the tokens to the answer with a given tokenizer"""
    answer_tokens = tokenizer.convert_ids_to_tokens(
        token, skip_special_tokens=True)
    return tokenizer.convert_tokens_to_string(answer_tokens)


input_ids, attention_mask = encode(tokenizer,"Hello, my dog is cute")
print(input_ids, attention_mask)
a = model(torch.tensor(
            [input_ids]), attention_mask=torch.tensor([attention_mask]))
ans_tokens= input_ids[4]
answer = decode(tokenizer,ans_tokens)

[0, 20920, 6, 127, 2335, 16, 11962, 2] [1, 1, 1, 1, 1, 1, 1, 1]


In [58]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("hoanhkhoa/roberta-base-finetuned-ner")
model = AutoModelForTokenClassification.from_pretrained("hoanhkhoa/roberta-base-finetuned-ner")

nlp = pipeline("ner", model=model, tokenizer=tokenizer)
example = "My name is Sarah and I live in London"

ner_results = nlp(example)
print(ner_results)

[{'entity': 'LABEL_0', 'score': 0.9998253, 'index': 1, 'word': 'ĠMy', 'start': 0, 'end': 2}, {'entity': 'LABEL_0', 'score': 0.9998684, 'index': 2, 'word': 'Ġname', 'start': 3, 'end': 7}, {'entity': 'LABEL_0', 'score': 0.99989426, 'index': 3, 'word': 'Ġis', 'start': 8, 'end': 10}, {'entity': 'LABEL_1', 'score': 0.9957847, 'index': 4, 'word': 'ĠSarah', 'start': 11, 'end': 16}, {'entity': 'LABEL_0', 'score': 0.9998487, 'index': 5, 'word': 'Ġand', 'start': 17, 'end': 20}, {'entity': 'LABEL_0', 'score': 0.9998658, 'index': 6, 'word': 'ĠI', 'start': 21, 'end': 22}, {'entity': 'LABEL_0', 'score': 0.99993086, 'index': 7, 'word': 'Ġlive', 'start': 23, 'end': 27}, {'entity': 'LABEL_0', 'score': 0.999918, 'index': 8, 'word': 'Ġin', 'start': 28, 'end': 30}, {'entity': 'LABEL_5', 'score': 0.99845695, 'index': 9, 'word': 'ĠLondon', 'start': 31, 'end': 37}]


In [59]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("slauw87/bart_summarisation")
model = AutoModelForTokenClassification.from_pretrained("slauw87/bart_summarisation")

nlp = pipeline("ner", model=model, tokenizer=tokenizer)
example = "My name is Sarah and I live in London"

ner_results = nlp(example)
print(ner_results)

Downloading: 100%|█████████████████████████████| 300/300 [00:00<00:00, 63.1kB/s]
Downloading: 100%|██████████████████████████| 1.59k/1.59k [00:00<00:00, 311kB/s]
Downloading: 100%|███████████████████████████| 780k/780k [00:00<00:00, 4.99MB/s]
Downloading: 100%|███████████████████████████| 446k/446k [00:00<00:00, 3.52MB/s]
Downloading: 100%|█████████████████████████| 1.29M/1.29M [00:00<00:00, 7.44MB/s]
Downloading: 100%|█████████████████████████████| 239/239 [00:00<00:00, 47.7kB/s]


ValueError: Unrecognized configuration class <class 'transformers.models.bart.configuration_bart.BartConfig'> for this kind of AutoModel: AutoModelForTokenClassification.
Model type should be one of YosoConfig, NystromformerConfig, QDQBertConfig, FNetConfig, LayoutLMv2Config, RemBertConfig, CanineConfig, RoFormerConfig, BigBirdConfig, ConvBertConfig, IBertConfig, MobileBertConfig, DistilBertConfig, AlbertConfig, CamembertConfig, XLMRobertaXLConfig, XLMRobertaConfig, MegatronBertConfig, MPNetConfig, LongformerConfig, RobertaConfig, DebertaV2Config, DebertaConfig, FlaubertConfig, SqueezeBertConfig, BertConfig, GPT2Config, XLNetConfig, XLMConfig, ElectraConfig, FunnelConfig, LayoutLMConfig, Data2VecTextConfig.

In [31]:
input_ids

[0, 20920, 6, 127, 2335, 16, 11962, 2]

In [42]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer

def get_model(model):
  """Loads model from Hugginface model hub"""
  try:
    model = AutoModelForQuestionAnswering.from_pretrained(model)
    model.save_pretrained('./model')
  except Exception as e:
    raise(e)

def get_tokenizer(tokenizer):
  """Loads tokenizer from Hugginface model hub"""
  try:
    tokenizer = AutoTokenizer.from_pretrained(tokenizer)
    tokenizer.save_pretrained('./model')
  except Exception as e:
    raise(e)

get_model('mrm8488/mobilebert-uncased-finetuned-squadv2')
get_tokenizer('mrm8488/mobilebert-uncased-finetuned-squadv2')

In [25]:
!pip3 list

Package                  Version  
------------------------ ---------
absl-py                  0.15.0   
aiohttp                  3.8.1    
aiosignal                1.2.0    
anyio                    3.5.0    
appnope                  0.1.2    
asgiref                  3.5.0    
asttokens                2.0.5    
astunparse               1.6.3    
async-timeout            4.0.2    
attrs                    21.4.0   
backcall                 0.2.0    
cachetools               5.0.0    
certifi                  2021.10.8
charset-normalizer       2.0.12   
click                    8.0.4    
cycler                   0.11.0   
debugpy                  1.5.1    
decorator                5.1.1    
entrypoints              0.4      
executing                0.8.3    
fastapi                  0.75.0   
filelock                 3.6.0    
flatbuffers              1.12     
fonttools                4.31.2   
frozenlist               1.3.0    
fsspec                   2022.2.0 
gast                

In [21]:
!pip3 uninstall transformers == 4.18.0

Uninstalling transformers-3.4.0:
  Would remove:
    /Users/parthshah/Documents/Northeastern/Spring2022/BigDataAnalytics/Assignment3/.venv/bin/transformers-cli
    /Users/parthshah/Documents/Northeastern/Spring2022/BigDataAnalytics/Assignment3/.venv/lib/python3.8/site-packages/transformers-3.4.0.dist-info/*
    /Users/parthshah/Documents/Northeastern/Spring2022/BigDataAnalytics/Assignment3/.venv/lib/python3.8/site-packages/transformers/*
Proceed (y/n)? ^C
ERROR: Operation cancelled by user


In [17]:
!pip3 install transformers==3.4.0

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  Using cached https://files.pythonhosted.org/packages/2c/4e/4f1ede0fd7a36278844a277f8d53c21f88f37f3754abf76a5d6224f76d4a/transformers-3.4.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/f8/30/a47fb2ab4b1b214f3177efe700af7032b4ac229edae148a4f06016c8541a/sentencepiece-0.1.96-cp38-cp38-macosx_10_6_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/f3/fe/cb38ae6a20143da7bf5427b1243fb6ccd238a0ca15393cf111ecc0176da8/tokenizers-0.9.2-cp38-cp38-macosx_10_11_x86_64.whl
  Found existing installation: tokenizers 0.11.6
    Uninstalling tokenizers-0.11.6:
      Successfully uninstalled tokenizers-0.11.6
  Found existing installation: tran

In [24]:
!pip3 install transformers==4.18.0

  Using cached https://files.pythonhosted.org/packages/8f/e9/c2b4c823b3959d475a570c1bd2df4125478e2e37b96fb967a87933ae7134/transformers-4.18.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ab/c2/34e886390a065b463bfa435426d990a50d2184fedab4e034a2dc8e37329d/tokenizers-0.11.6-cp38-cp38-macosx_10_11_x86_64.whl
  Found existing installation: tokenizers 0.9.2
    Uninstalling tokenizers-0.9.2:
      Successfully uninstalled tokenizers-0.9.2
  Found existing installation: transformers 3.4.0
    Uninstalling transformers-3.4.0:
      Successfully uninstalled transformers-3.4.0
You should consider upgrading via the 'pip install --upgrade pip' command.
